# Import Necessary Function and Library

In [15]:
import minsearch
import json
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [16]:
index = minsearch.Index(
    text_fields = {'question','text','section'},
    keyword_fields=['course']
)
index.fit(documents)
boost = {'question':3.0, 'section':0.5}

In [40]:
def search(query):
    boost = {'question':3.0, 'section':0.5}
    search_results = index.search(
        query=query,
        boost_dict = boost,
        filter_dict = {'couse':'data-engineering-zoomcamp'},
        num_results=5
    )
    return search_results
def build_prompt(query,search_results):
    prompt_template = """
    You are Course Assistant. You answer the question based on context. Use only fact the CONTEXT for answering the question.
    
    QUESTION: {question}
    CONTEXT : 
    {context}
    """.strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query,context=search_results).strip()
    return prompt
def llm_open_api(prompt):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(model='gpt-3.5-turbo-16k',messages = [{'role':'user',"content":prompt}])
    return response.choices[0].message.content

In [36]:
from openai import OpenAI
import sys,os,os.path

In [37]:
api_key = os.environ['OPEN_AI_API_KEY']
client = OpenAI(api_key=api_key)

In [38]:
def llm_open_api(prompt):
    client = OpenAI(api_key=api_key)
    response = client.chat.completions.create(model='gpt-3.5-turbo-16k',messages = [{'role':'user',"content":prompt}])
    return response.choices[0].message.content

In [39]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm_open_api(prompt)
    return answer

In [41]:
rag('the course already started. Can I still enroll the course?')

"Yes, you can still enroll in the course. Even though the course has already started, you can still participate and take part in the course activities. However, if you join late, you may not be able to submit some of the homework assignments. To be eligible for a certificate, you will need to submit 2 out of 3 course projects and review 3 peers' projects by the designated deadline."

# Use Ollama to Run Your LLM

In [42]:
from openai import OpenAI

In [49]:
def llm_ollama(prompt):
    client = OpenAI(
    base_url = 'http://localhost:11434/v1/',api_key = 'ollama',)
    response = client.chat.completions.create(model='phi3',messages = [{'role':'user',"content":prompt}])
    return response.choices[0].message.content

In [51]:
def rag_ollama(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm_ollama(prompt)
    return answer

In [52]:
rag_ollama('the course already started. Can I still enroll the course?')

" Yes, you can still enroll in the machine learning Zoom camp even if the course has already started. However, you won't be able to submit some of the homeworks. To receive a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline."

# Search with ElasticSearch

In [53]:
from elasticsearch import Elasticsearch

In [56]:
es_client = Elasticsearch('http://127.0.0.1:9200')

In [57]:
es_client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))))